Write code to classify two famous celebrities:
We need to prepare the data to do so and also prepare the model as well.

In [54]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [55]:
class MyCLRuleMonitor(tf.keras.callbacks.Callback):
  def __init__(self, CL):
    super(MyCLRuleMonitor).__init__()
    self.CL = CL

  def on_epoch_end(self, epoch, logs=None):
    trainScore = logs["accuracy"]
    testScore = logs["val_accuracy"]

    if testScore > trainScore and testScore >= self.CL:
      self.model.stop_training = True

Steps for Model creation:
0. Data Preprocessing
    i. Image Generators
    ii. Splitting Train and Test data from Generators
1. Model architecting + Prepro
    i. CNN + ANN layers creation
2. Model Compilation
3. Model Training 
4. Model Evaluation
5. Model Testing / Deploy

In [56]:
# ImageGenerators:
training_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
testing_image_generator= tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.0)

In [57]:
training_image_data = training_image_generator.flow_from_directory('celeb/data/training',
                                                                   batch_size = 20,
                                                                   class_mode = 'binary',
                                                                   target_size= (128,128))

testing_image_data = testing_image_generator.flow_from_directory('celeb/data/validation',
                                                                   batch_size = 20,
                                                                   class_mode = 'binary',
                                                                   target_size= (128,128))

Found 31 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


In [58]:
# Model creation:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3,3), input_shape = training_image_data.image_shape , activation = 'relu', padding = 'same'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Conv2D(32, (3,3), padding = 'same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Flatten())

# FC:
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))

model.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

model.summary()

c:\Users\thedo\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 64, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 128)            │     4,194,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,254,241 (16.23 MB)

 Trainable params: 4,254,241 (16.23 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics= ['accuracy'])

In [62]:
model.fit(training_image_data, 
          validation_data= testing_image_data,
          epochs = 1000,
          validation_steps= (len(testing_image_data.filenames) // testing_image_data.batch_size),
          steps_per_epoch = (len(training_image_data.filenames) // training_image_data.batch_size),
          callbacks = MyCLRuleMonitor(0.7))

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 1.0000 - loss: 3.9088e-08 - val_accuracy: 0.6500 - val_loss: 2.7489
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 3.3853e-08 - val_accuracy: 0.7000 - val_loss: 2.6931
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 2.3150e-08 - val_accuracy: 0.7500 - val_loss: 1.4662
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 2.7468e-08 - val_accuracy: 0.6500 - val_loss: 2.6742
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 1.1090e-08 - val_accuracy: 0.6500 - val_loss: 2.6135
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 2.7049e-08 - val_accuracy: 0.7000 - val_loss: 2.5845
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 1.5632e-08 - val_accuracy: 0.7500 - val_loss: 1.7826
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 1.8788e-08 - val_accu

In [ ]:
# Testing | Deployment
# Read the image - convert to np array - add dimention using np - get prediction - change prediction aproximation to value

image = tf.keras.preprocessing.image.load_img('celeb/data/validation/salman/salman-khan_1.jpg', target_size = (128,128))

image_arr = tf.keras.preprocessing.image.img_to_array(image)

np_image_arr = np.expand_dims(image_arr, axis = 0)

predictions = model.predict(np_image_arr)
print(predictions)

# 2. Find the index of the class with the highest probability
predicted_class_index = np.argmax(predictions) #

# print(predicted_class_index)
# print(training_image_data.classes)
# 3. Create a reverse mapping dictionary to get the class name from the index
# Original: {'class_name': index} -> Reversed: {index: 'class_name'}
class_names_map = {v: k for k, v in training_image_data.class_indices.items()}

# print(class_names_map)
# 4. Use the index to look up the actual class name
predicted_class_name = class_names_map[predicted_class_index]

# 5. Print the result
print(f"The predicted celeb name is: {predicted_class_name}") #

# {0: 'salman', 1: 'shahrukh'}

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
[[1.]]
The predicted celeb name is: salman
